In [1]:
from constants import *
import csv
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os
from emotion_recognition import EmotionRecognition
import cv2
import sys

C:\Users\x-cla\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
cascade_classifier = cv2.CascadeClassifier(CASC_PATH)

#format jpg/png file to fit into network size
def format_image(image):
    image = np.asarray(image)
    if len(image.shape) > 2 and image.shape[2] == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        image = cv2.imdecode(image, cv2.IMREAD_GRAYSCALE)
    faces = cascade_classifier.detectMultiScale(
        image,
        scaleFactor=1.3,
        minNeighbors=5
    )
    # None is we don't found an image
    if not len(faces) > 0:
        return None
    max_area_face = faces[0]
    for face in faces:
        if face[2] * face[3] > max_area_face[2] * max_area_face[3]:
            max_area_face = face
    # Chop image to face
    face = max_area_face
    image = image[face[1]:(face[1] + face[2]), face[0]:(face[0] + face[3])]
    # Resize image to network size
    try:
        image = cv2.resize(image, (SIZE_FACE, SIZE_FACE),
                           interpolation=cv2.INTER_CUBIC) / 255.
    except Exception:
        print("[+] Problem during resize")
        return None

    return image

#format jpg/png file (without normalization)
def form(image):
    image = np.asarray(image)
    if len(image.shape) > 2 and image.shape[2] == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        image = cv2.imdecode(image, cv2.IMREAD_GRAYSCALE)
    faces = cascade_classifier.detectMultiScale(
        image,
        scaleFactor=1.3,
        minNeighbors=5
    )
    # None is we don't found an image
    if not len(faces) > 0:
        return None
    max_area_face = faces[0]
    for face in faces:
        if face[2] * face[3] > max_area_face[2] * max_area_face[3]:
            max_area_face = face
    # Chop image to face
    face = max_area_face
    image = image[face[1]:(face[1] + face[2]), face[0]:(face[0] + face[3])]
    # Resize image to network size
    try:
        image = cv2.resize(image, (SIZE_FACE, SIZE_FACE),
                           interpolation=cv2.INTER_CUBIC)
    except Exception:
        print("[+] Problem during resize")
        return None

    return image

#im = Image.open("0.jpg")
#hi = form(im)

In [3]:
def infer(image):
    
    #Load model
    network = EmotionRecognition()
    network.build_network()
    
    #Predict
    result = network.predict(format_image(image))
    label = np.argmax(result[0])
    print("Emotion: " + str(EMOTIONS[np.argmax(result[0])]))
    return label

#im = Image.open("0.jpg")
#infer(im)

In [4]:
def collect(image):
    #Get predicted label
    labell = infer(image)
    
    #Change image to string of values
    data = form(image)
    l=data.tolist()
    new = []
    for sublist in l:
        for item in sublist:
            new.append(item)
    stri=' '.join([str(int(i)) for i in new])
    
    #Add to csv file
    row=[str(labell), stri,'Training' ]
    with open("./collected/collect.csv", 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)

#im = Image.open("0.jpg")
#collect(im)

[+] Building CNN
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Emotion: fearful


In [ ]:
def reframe():
    
    #Get data from collect.csv and add it to dataset
    with open("./collected/collect.csv", newline = '') as f_out, open("./data/test.csv", 'a', newline='') as f_in:
        reader = csv.reader(f_out)
        next(reader) #skip first row
        writer = csv.writer(f_in)
        for row in reader:
            writer.writerow(row)

In [ ]:
def train():
    os.system("python3 emotion_recognition.py")